In [39]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ChatMessageHistory
import json
from langchain.prompts import PromptTemplate
from langchain.schema import SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.memory import ConversationBufferMemory



In [32]:
from langfuse.callback import CallbackHandler
handler = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))

In [33]:
GENERATIONS_PATH = "../datasets/manual_gpt3_50_50_generations_merged.csv"

In [45]:
data = pd.read_csv(GENERATIONS_PATH)

data["eval"] = data["eval"].apply(lambda x: x.replace('{"completion":', "").replace('}', "").replace('"',""))
data["completion"] = data["completion"].apply(lambda x: x.replace('{"completion":', "").replace('}', "").replace('"',""))
whole_prompt = json.loads(data["prompt"][0])[0][0]["content"]
user_prompt = "Q:" + whole_prompt.split("Q:")[-1]
ai_answer = data["completion"][0]

conversation = user_prompt + ai_answer
print(conversation)
# data

Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Traveler's Lined Notebook Refill - Regular Size
Description: Think about the days you went to school. Based on people’s past experience as a student, we receive comments saying that this is the most easy type of notebook to use. The lines are printed with an interval of 6.5mm, and the paper is our originally m
Product 2: 
Title: Traveler's Company Traveler's Company Notebook Accessories
Description: none
A:
These two products are not the same because they have different titles and descriptions, and one refers to a specific refill for a notebook while the other refers to a general category of notebook accessories.


In [68]:


task = """Now output your decision as a simple 'Match' or 'No Match'. 
Also provide a similarity score for the items and a confidence score for your decision as percentage values, 100% referring to perfect similarity or full confidence. 
Further specify which attributes and values had a high influence on your decision.
Please use the following format which includes an example output:
Decision: Match
Similarity: 75%
Confidence: 60%
Important Attributes: brand, serial number"""



STEP_CONFIDENCE_PROMPT = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI."),
            ("human", "{user_prompt}"),
            ("ai", "{ai_answer}"),
            ("human", task),
        ])


print(STEP_CONFIDENCE_PROMPT.format(user_prompt=user_prompt, ai_answer=ai_answer))



System: You are a helpful AI.
Human: Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Traveler's Lined Notebook Refill - Regular Size
Description: Think about the days you went to school. Based on people’s past experience as a student, we receive comments saying that this is the most easy type of notebook to use. The lines are printed with an interval of 6.5mm, and the paper is our originally m
Product 2: 
Title: Traveler's Company Traveler's Company Notebook Accessories
Description: none
A:

AI: These two products are not the same because they have different titles and descriptions, and one refers to a specific refill for a notebook while the other refers to a general category of notebook accessories.
Human: Now output your decision as a simple 'Match' or 'No Match'. 
Also provide a similarity score for the items and a confidence score for your decision as percentage values, 100% referring to perfect similarity or full confidence. 
Further

In [69]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0301", temperature=0, max_tokens=50)

history = ChatMessageHistory()


chain = LLMChain(llm=llm, prompt=STEP_CONFIDENCE_PROMPT, callbacks=[handler])

chain.run(user_prompt=user_prompt, ai_answer=ai_answer, callbacks=[handler])


